# Welcome to BASS!

One Click HRV

This notebok was custom designed by Abigail Dobyns for Dr. Curras-Collazo's lab.

    BASS: Biomedical Analysis Software Suite for event detection and signal processing.
    Copyright (C) 2015  Abigail Dobyns

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>

## Initalize

Run the following code block to intialize the program.

*run this block* **one** *time*

In [1]:
from bass import *

BASS ready!


# Begin User Input

For help, check out the wiki: [Protocol](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol)

Or the video tutorial: Coming Soon!

## Load Data File
Use the following block to change your settings. You **must** use this block.

Here are some helpful information about the loading settings:

**Settings['folder']= Full File Path for data input:**
Designate the path to your file to load. It can also be the relative path to the folder where this notebook is stored. This does not include the file itself.

    Mac OSX Example: '/Users/MYNAME/Documents/bass'
    Microsoft Example: r'C:\\Users\MYNAME\Documents\bass'
    

**Settings['Label']= File name:**
This is the name of your data file. It should include the file type. This file should NOT have a header and the first column must be time in seconds. Note: This file name will also appear as part of the output files names.

    'rat34_ECG.txt' 

**Settings['Output Folder'] = Full File Path for data output:** Designate the location of the folder where you would like the folder containing your results to go. If the folder does not exist, then it will be created. A plots folder, called 'plots' will be created inside this folder for you if it does not already exist. 

    Mac OSX Example: '/Users/MYNAME/Documents/output'
    Microsoft Example: r'C:\\Users\MYNAME\Documents\output' 

[Loading a file](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#load)

**WARNING** All text input should be raw, especially if in Windows.
    
    r'string!'
    r"string!"
    
## Other Settings

For more information about other settings, go to: 

[Transforming Data](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#enter-your-settings-for-data-transformation)

[Baseline Settings](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#set-baseline-for-thresholding)

[Peak Detection Settings](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#peak-detection-settings)

[Burst Detection Settings](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#enter-burst-settings)

In [2]:
#initialize new file
Data = {}
Settings = {}
Results ={}

############################################################################################
#manual Setting block
Settings['folder']= r"/Users/abigaildobyns/Downloads"
Settings['Label'] = r'2014-06-10rat70-ECG.txt'
Settings['Output Folder'] = r"/Users/abigaildobyns/Desktop/2014-06-10rat70-ECG"

#transformation Settings
Settings['Absolute Value'] = True #Must be True if Savitzky-Golay is being used

Settings['Bandpass Highcut'] = 100 #in Hz
Settings['Bandpass Lowcut'] = 30 #in Hz
Settings['Bandpass Polynomial'] = 4 #integer

Settings['Linear Fit'] = False #between 0 and 1 on the whole time series
Settings['Linear Fit-Rolling R'] = 0.75 #between 0 and 1
Settings['Linear Fit-Rolling Window'] = 1000 #window for rolling mean for fit, unit is index not time
Settings['Relative Baseline'] = 0 #default 0, unless data is normalized, then 1.0. Can be any float

Settings['Savitzky-Golay Polynomial'] = 4 #integer
Settings['Savitzky-Golay Window Size'] = 301 #must be odd. units are index not time

#Baseline Settings
Settings['Baseline Type'] = r'static' #'linear', 'rolling', or 'static'
#For Linear
Settings['Baseline Start'] = 0.0 #start time in seconds
Settings['Baseline Stop'] = 1.0 #end time in seconds
#For Rolling
Settings['Rolling Baseline Window'] = r'none' #leave as 'none' if linear or static

#Peaks
Settings['Delta'] = 0.03
Settings['Peak Minimum'] = 0.02 #amplitude value
Settings['Peak Maximum'] = 0.2  #amplitude value

#Bursts
Settings['Burst Area'] = False #calculate burst area
Settings['Exclude Edges'] = True #False to keep edges, True to discard them
Settings['Inter-event interval minimum (seconds)'] = 0.0100 #only for bursts, not for peaks
Settings['Maximum Burst Duration (s)'] = 10 
Settings['Minimum Burst Duration (s)'] = 0
Settings['Minimum Peak Number'] = 1 #minimum number of peaks/burst, integer
Settings['Threshold']= 0.025 #linear: proportion of baseline. 
                           #static: literal value.
                           #rolling, linear ammount grater than rolling baseline at each time point.

#Outputs
Settings['Generate Graphs'] = False #create and save the fancy graph outputs


#Settings that you should not change unless you are a super advanced user:
#These are settings that are still in development
Settings['Graph LCpro events'] = False
Settings['File Type'] = r'Plain' #'LCPro', 'ImageJ', 'SIMA', 'Plain', 'Morgan'
Settings['Milliseconds'] = False
############################################################################################
##DO NOT TYPE BELOW THIS LINE
############################################################################################

def analyze(Data, Settings, Results):
    """
    The pipeline for event detection. Follows the strict '3 arguments in, 3 arguments out'
    rule. These dictionaries must be intalized before this, as well as all of the Settings values.
    Detects bursts and peaks for the data file that is uploaded.
    Settings used are saved out automatically with a time stamp as a receipt of each analysis performed.
    This version of analyze was made specifically for the UCSD HRV group.

    Parameters
    ----------
    Data: dictionary
        an empty dictionary named Data.
    Settings: dictionary
        dictionary that contains the user's settings.
    Results: dictionary
        an empty dictionary named Results.
    
    Returns
    -------
    Data: dictionary
        Contains the DataFrames with the time series data. Keys define which version of the data it is.
    Settings: dictionary
        dictionary that contains the user's settings.
    Results: dictionary
        Contains the following objects:
            Peaks: dictionary
                keys are the column names from the Data DataFrames. objects are DataFrames that contain information about each peak detected, indexed by peak time.
            Peaks-Master: DataFrame
                multi-indexed DataFrame, created by concatenating all Peaks DataFrames. Column names and peak time are the two indexes. Automatically saved in the Settings['output folder'] location.
            Bursts: dictionary
                keys are the column names from the Data DataFrames. objects are DataFrames that contain information about each burst detected. has an arbitrary index, which can be roughly thought of as burst number.
            Bursts-Master: DataFrame
                multi-indexed DataFrame, created by concatenating all Bursts DataFrames. Column names and burst number are the two indexes. Automatically saved in the Settings['output folder'] location.
            

    Notes
    -----
    This function is the top level function of the bass pipeline. 
    It has a few handy printed outputs, such as how long an analysis took, which step was just completed, lists of which objects contained no peaks or bursts. it also prints a list of key names and analysis measurements, which can be used in further analysis steps.
    
    """
    start = t.clock()
    #Load
    Data, Settings = load_wrapper(Data, Settings)
    
    Data['original'].columns = [Settings['Label']]
    
    #For this data set, we know the frame rate
    frames = np.round(1/Settings['Sample Rate (s/frame)'])
    Settings['Sample Rate (s/frame)'] = 1/float(frames)
    print "Rounded Sampling Rate (s/frame): %s" %Settings['Sample Rate (s/frame)']

    #transform data
    Data, Settings = transform_wrapper(Data, Settings)
    print 'Transformation completed'

    #set baseline
    Data, Settings, Results = baseline_wrapper(Data, Settings, Results)
    print 'Baseline set completed'

    #run peak detection
    Results = event_peakdet_wrapper(Data, Settings, Results)
    print 'Peak Detection completed'

    #run burst detection
    Results = event_burstdet_wrapper(Data, Settings, Results)
    print 'Burst Detection completed'

    #Save all the graphs
    if Settings['Generate Graphs'] == True:
        for label, col in Data['original'].iteritems():
            graph_detected_events_save(Data, Settings, Results, 
                                  roi = label, lcpro = Settings['Graph LCpro events'])
        print "Graphs Saved"

    #Save master files 
    Results['Peaks-Master'].to_csv(r'%s/%s_Peak_Results.csv'
                                   %(Settings['Output Folder'], Settings['Label']))
    Results['Bursts-Master'].to_csv(r'%s/%s_Bursts_Results.csv'
                                    %(Settings['Output Folder'], Settings['Label']))

    #Save Master Summary Files
    burst_grouped = Results['Bursts-Master'].groupby(level=0)
    burst_grouped = burst_grouped.describe()
    burst_grouped.to_csv(r'%s/%s_Bursts_Results_Summary.csv'
                                           %(Settings['Output Folder'], Settings['Label']))
    
    peak_grouped = Results['Peaks-Master'].groupby(level=0)
    peak_grouped= peak_grouped.describe()
    peak_grouped.to_csv(r'%s/%s_Peaks_Results_Summary.csv'
                                           %(Settings['Output Folder'], Settings['Label']))

    #save settings
    Settings_panda = DataFrame.from_dict(Settings, orient='index')
    now = datetime.datetime.now()
    colname = 'Settings: ' + str(now)
    Settings_panda.columns = [colname]
    Settings_panda = Settings_panda.sort()
    Settings_panda.to_csv(r"%s/%s_Settings_%s.csv"%(Settings['Output Folder'], 
                                                 Settings['Label'], 
                                                 now.strftime('%Y_%m_%d__%H_%M_%S')))

    end = t.clock()
    run_time = end-start
    print "Analysis Complete: ", np.round(run_time,4), " Seconds"
    
    print "\n--------------------------------------------"

    print "Data Column Names/Keys"
    print "-----"
    for name in Data['original']:
        print name
    print "\n--------------------------------------------"
    print "Available Measurements from Peaks for further analysis:"
    print "-----"
    for label, col in Results['Peaks-Master'].iteritems():
        print label
    print "\n--------------------------------------------"
    print "Available Measurements from Bursts for further analysis:"
    print "-----"
    for label, col in Results['Bursts-Master'].iteritems():
        print label
    
    print "\n---------------------------"
    print '|Event Detection Complete!|'
    print "---------------------------\n"
    return Data, Settings, Results

def psd_event(event_type, meas, key, scale, Data, Settings, Results):
    '''
    Wrapper that plots the power spectral density of one column's event measurement by calling scipy.signal.welch
    The measurments must first be interpolated so that they can be handled like a regularly sampled descrete series.
    User can choose the scale calling either 'raw' or 'db'.
    the varibles inside this call are set to functions of the sampling rate of the time series. 
    nperseg and nfft are 256, noverlap is 128. scale defaults to raw.
    
    Parameters
    ----------
    Data: dictionary
        must contain Data['original']
    Settings: dictionary
        dictionary that contains the user's settings. requires Settings['Sample Rate (s/frame)']
    Results: dictionary
        an dictionary named Results.
    
    Returns
    -------
    Results: dictionary
        Updated to contains the following objects if bands are specified:
        Results['PSD-Events']: a dictionary where each key is a dataframe with area in band data
        
    Notes
    -----
    raw plots in units of V**2/Hertz. db plots in units of dB/Hertz. the conversion is dB = 10*log10(raw)
    hvaing varibles from the psd call as functions of the sampling rate of the signal is an appropreate way to handle the variblity of data collected.

    changing the interpolation frequency to something appropreate for you species and measurement is critical! this can drastically change the 
    Examples
    --------
    #These settings are for human heart rate

    Settings['PSD-Event'] = Series(index = ['Hz','ULF', 'VLF', 'LF','HF','dx'])
    #Set PSD ranges for power in band

    Settings['PSD-Event']['hz'] = 4.0 #freqency that the interpolation and PSD are performed with.
    Settings['PSD-Event']['ULF'] = 0.03 #max of the range of the ultra low freq band. range is 0:ulf
    Settings['PSD-Event']['VLF'] = 0.05 #max of the range of the very low freq band. range is ulf:vlf
    Settings['PSD-Event']['LF'] = 0.15 #max of the range of the low freq band. range is vlf:lf
    Settings['PSD-Event']['HF'] = 0.4 #max of the range of the high freq band. range is lf:hf. hf can be no more than (hz/2)
    Settings['PSD-Event']['dx'] = 10 #segmentation for the area under the curve. 

    event_type = 'Peaks'
    meas = 'Intervals'
    key = 'Mean1'
    scale = 'raw'
    
    Results = psd_event(event_type, meas, key, scale, Data, Settings, Results)
    Results['PSD-Event'][key]

    References
    ----------
    http://docs.scipy.org/doc/scipy-dev/reference/generated/scipy.signal.welch.html
    interpolation adopted form Rhenan Bartels Ferreira 
    https://github.com/RhenanBartels/biosignalprocessing/blob/master/psdRRi.py
    '''
    try:
        psd_e_folder = Settings['Output Folder'] +"/PSD-Event"
        mkdir_p(psd_e_folder) #makes a plots folder if does not exist
        
    except:
        try:
            psd_e_folder = Settings['Output Folder'] +"\PSD-Event"
            mkdir_p(psd_e_folder) #makes a plots folder if does not exist
            
        except:
            print "Could not make psd events folder. :("

    if event_type.lower() == 'peaks':
        measurement = Results['Peaks']
        columns = Results['Peaks-Master']

    elif event_type.lower() == 'bursts':
        measurement = Results['Bursts']
        columns = Results['Bursts-Master']
    else:
        raise ValueError('Not an acceptable event type measurement.\n Must be "Peaks" or "Bursts" ')


    freq_list = measurement[key][meas].tolist()
    time_list = measurement[key].index.tolist()

    if freq_list[-1] != freq_list[-1]: #check if NaN
        freq_list = freq_list[:-1]
        time_list = time_list[:-1]
        
    #for this group, convert to niHR which is in intervals/min
    '''
    y = []
    for x in freq_list:
        x = 60/x
        y.append(x)
    freq_list = y
    '''
    #evenly spaced array using interpolation hertz        
    
    tx = np.arange(time_list[0], time_list[-1], (1.0/(Settings['PSD-Event']['hz'])))
   
    #interpolate!
    tck = scipy.interpolate.splrep(time_list, freq_list, s = 0)
    
    freq_x = scipy.interpolate.splev(tx, tck, der = 0)
    
    #Number os estimations
    P = int((len(tx) - 256 / 128)) + 1 #AD doesn't know what this does, but i dare not touch a damn thing.
    
    hertz = int(Settings['PSD-Event']['hz'])


    Fxx, Pxx = scipy.signal.welch(freq_x, fs = hertz, window="hanning", 
                                  nperseg=256, noverlap=128, nfft=256, 
                                  detrend='linear', return_onesided=True, 
                                  scaling='density')
    
    if scale.lower() == 'db':
        plt.plot(Fxx,10*np.log10(Pxx))
        plt.ylabel('Power (dB/Hz)')
        plt.title("Welch's PSD of %s in dB/Hz" %(key))

    else:
        plt.plot(Fxx, Pxx)
        plt.ylabel(r"PSD $(s^2$/Hz)")
        plt.title(r"Welch's PSD of %s- %s in (s^2/Hz)" %(meas, key))
    
    plt.xlabel("Frequency (Hz)")
    
    try:
        plt.savefig(r'%s/%s-%s PSD - %s.pdf'%(psd_e_folder, event_type,meas, key))
    except:
        plt.savefig(r'%s\%s-%s PSD - %s.pdf'%(psd_e_folder, event_type,meas, key))
    
    #plt.show()
    

    
    try:
        if 'PSD-Event'not in Results.keys():
            Results['PSD-Event'] = {}
        
        if key not in Results['PSD-Event'].keys():
            Results['PSD-Event'][key] = DataFrame(index = ['ULF', 'VLF', 'LF','HF',
                                                           'LF/HF', 'Total', 'Scale'])
        
        df_power = DataFrame( index = Fxx, data = Pxx)
        df_power.columns = ['Power']
        results_psd = Series(index = ['ULF', 'VLF', 'LF','HF','LF/HF', 'Total' 'Scale'])

        #ULF
        df_ulf = df_power[df_power.index<Settings['PSD-Event']['ULF']]
        results_psd['ULF'] = scipy.integrate.simps(df_ulf['Power'], 
                                                   df_ulf.index, 
                                                   dx =Settings['PSD-Event']['dx'])

        #VLF
        df_vlf = df_power[(df_power.index>Settings['PSD-Event']['ULF']) & (df_power.index<=Settings['PSD-Event']['VLF'])]
        results_psd['VLF'] = scipy.integrate.simps(df_vlf['Power'], df_vlf.index, dx =Settings['PSD-Event']['dx'])

        #LF
        df_lf = df_power[(df_power.index>Settings['PSD-Event']['VLF']) & (df_power.index<=Settings['PSD-Event']['LF'])]
        results_psd['LF'] = scipy.integrate.simps(df_lf['Power'], df_lf.index, dx =Settings['PSD-Event']['dx'])

        #HF
        df_hf = df_power[(df_power.index>Settings['PSD-Event']['LF']) & (df_power.index<=Settings['PSD-Event']['HF'])]
        results_psd['HF'] = scipy.integrate.simps(df_hf['Power'], df_hf.index, dx =Settings['PSD-Event']['dx'])
        
        #HF
        df_t = df_power[df_power.index<=Settings['PSD-Event']['HF']]
        results_psd['Total'] = scipy.integrate.simps(df_t['Power'], df_t.index, 
                                                  dx =Settings['PSD-Event']['dx'])

        #LF/HF
        results_psd['LF/HF'] = results_psd['LF']/results_psd['HF']
        results_psd['Scale'] = NaN
        
        if scale.lower() == 'db':
            try:
                results_psd = 10*np.log10(results_psd)
                results_psd['Scale'] = 'dB/Hz'
            except:
                results_psd['Scale'] = 's^2/Hz'
        else:
            results_psd['Scale'] = 's^2/Hz'
        
        Results['PSD-Event'][key][meas] = results_psd
        
        temp_psd_master = pd.concat(Results['PSD-Event'])
        try:
            temp_psd_master.to_csv(r'%s/%s_PSD_Events.csv' %(psd_e_folder, Settings['Label']))
            Settings['PSD-Event'].to_csv(r'%s/%s_PSD_Events_Settings.csv' %(psd_e_folder, Settings['Label']))
        except: #for windows
            temp_psd_master.to_csv(r'%s\%s_PSD_Events.csv' %(psd_e_folder, Settings['Label']))
            Settings['PSD-Event'].to_csv(r'%s\%s_PSD_Events_Settings.csv' %(psd_e_folder, Settings['Label']))
    except:
        print "Could not calculate power in band."
    return Results

pd.options.display.max_columns = 25

#Run detection
Data, Settings, Results = analyze(Data, Settings, Results)

#New HRV stuff
event_type = 'Peaks'
meas = 'Intervals'
key = Settings['Label']
start_time = t.clock()
#create results table
hrv = pd.DataFrame(columns=['Beats', 'Recording Length (s)','Mean HR', 'Mean RR', 
                            'STD RR', 'pNN50', 'rMSSD', 'IRRR', 'MADRR','TINN', 
                            'HRV index', 'ApEn', 'Sample Ent', 'HistEnt', 'SD1', 'SD2',
                            'ULF', 'VLF', 'LF', 'HF', 'Total Power', 'LF/HF Ratio'],
                  index = [key])

#total number of beats
try:
    hrv.ix[key]['Beats'] = Results['Peaks-Master']['Peaks Amplitude'].count()
except:
    hrv.ix[key]['Beats'] = NaN
    
#Mean Heart Rate, beats/min
try:
    t_sec = Data['trans'].index[-1]-Data['trans'].index[0]
    hrv.ix[key]['Recording Length (s)'] = t_sec
except:
    hrv.ix[key]['Recording Length (s)'] = NaN
    
try:
    t_min = t_sec/60
    hrv.ix[key]['Mean HR'] = hrv.ix[key]['Beats']/t_min
except:
    hrv.ix[key]['Mean HR'] = NaN
    
#mean r-r interval
try:
    hrv.ix[key]['Mean RR'] = Results['Peaks-Master']['Intervals'].mean()
except:
    hrv.ix[key]['Mean RR'] = NaN
#std r-r interval
try:
    hrv.ix[key]['STD RR'] = Results['Peaks-Master']['Intervals'].std()
except:
    hrv.ix[key]['STD RR'] = NaN
    
#PSD of frequency domain, in beats/min
Settings['PSD-Event'] = Series(index = ['Hz','ULF', 'VLF', 'LF','HF','dx'])
#Set PSD ranges for power in band

Settings['PSD-Event']['hz'] = 4.0 #freqency that the interpolation and PSD are performed with.
Settings['PSD-Event']['ULF'] = 0.03 #max of the range of the ultra low freq band. range is 0:ulf
Settings['PSD-Event']['VLF'] = 0.05 #max of the range of the very low freq band. range is ulf:vlf
Settings['PSD-Event']['LF'] = 0.15 #max of the range of the low freq band. range is vlf:lf
Settings['PSD-Event']['HF'] = 0.4 #max of the range of the high freq band. range is lf:hf. hf can be no more than (hz/2)
Settings['PSD-Event']['dx'] = 10 #segmentation for the area under the curve. 
scale = 'raw'

Results = psd_event(event_type, meas, key, scale, Data, Settings, Results)
plt.close()

try:
    hrv.ix[key]['ULF'] = Results['PSD-Event'][key].ix['ULF'][meas]
except:
    hrv.ix[key]['ULF'] = NaN
try:
    hrv.ix[key]['VLF'] = Results['PSD-Event'][key].ix['VLF'][meas]
except:
    hrv.ix[key]['VLF'] = NaN
try:    
    hrv.ix[key]['LF'] = Results['PSD-Event'][key].ix['LF'][meas]
except:
    hrv.ix[key]['LF'] = NaN
try:
    hrv.ix[key]['HF'] = Results['PSD-Event'][key].ix['HF'][meas]
except:
    hrv.ix[key]['HF'] = NaN
try:
    hrv.ix[key]['LF/HF Ratio'] = Results['PSD-Event'][key].ix['LF/HF'][meas]
except:
    hrv.ix[key]['LF/HF Ratio'] = NaN
try:
    hrv.ix[key]['Total Power'] = Results['PSD-Event'][key].ix['Total'][meas]
except:
    hrv.ix[key]['Total Power'] = NaN

#pNN50
try:
    rr_diffs = np.diff(Results[event_type][key][meas])
    rr_ints50 = rr_diffs[abs(rr_diffs)>0.05]
    pNN50 = 100*len(rr_ints50)/Results[event_type][key][meas].count()
    hrv.ix[key]['pNN50'] = pNN50
except:
    hrv.ix[key]['pNN50'] = NaN
    
#root mean square of successive differences (rMSSD)
try:
    hrv.ix[key]['rMSSD'] = np.sqrt(np.mean(Results[event_type][key][meas]**2))
except:
    hrv.ix[key]['rMSSD'] = NaN
#IRRR
try:
    rr_quants = scipy.stats.mstats.mquantiles(Results[event_type][key][meas])
    hrv.ix[key]['IRRR'] = rr_quants[-1] - rr_quants[0]
except:
    hrv.ix[key]['IRRR'] = NaN
#MADRR
try:
    hrv.ix[key]['MADRR'] = np.median(np.abs(Results[event_type][key][meas]))
except:
    hrv.ix[key]['MADRR'] = NaN
#TINN
#taken and adapted from gHRV

min_rr = Results[event_type][key][meas].min()
max_rr = Results[event_type][key][meas].max()
med_rr = (min_rr + max_rr)/2

#i believe that interval here is the sampling frequency, 
#based on gHRV documentation
try:
    interval = Data['original'].index[1]-Data['original'].index[0]
    lowhist = med_rr * interval * np.ceil((med_rr - min_rr)/interval)
    longhist = int(np.ceil((max_rr - lowhist)/interval) +1)
    vecthist=np.array([lowhist+interval*i for i in range(longhist)])
    h=np.histogram(Results[event_type][key][meas],vecthist)
    area = float(Results[event_type][key][meas].count()) * interval
    maxhist = max(h[0])
    hrv.ix[key]['TINN'] = area/maxhist
    #HRV index
    hrv.ix[key]['HRV index'] = Results[event_type][key][meas].count()/maxhist

except:
    hrv.ix[key]['TINN'] = NaN
    #HRV index
    hrv.ix[key]['HRV index'] = NaN
    
#Hist Ent
try:
    Results = histent_wrapper(event_type, meas, Data, Settings, Results)
    hrv.ix[key]['HistEnt'] = Results['Histogram Entropy'].ix[key][meas]
except:
    hrv.ix[key]['HistEnt'] = NaN

#poincare
try:
    Results = poincare_batch(event_type, meas, Data, Settings, Results)
    hrv.ix[key]['SD1'] =Results['Poincare SD1'].ix[key][meas]
    hrv.ix[key]['SD2'] =Results['Poincare SD2'].ix[key][meas]
except:
    hrv.ix[key]['SD1'] =NaN
    hrv.ix[key]['SD2'] =NaN
    
#Approximate Ent
try:
    Results = ap_entropy_wrapper(event_type, meas, Data, Settings, Results)
    hrv.ix[key]['ApEn'] = Results['Approximate Entropy'].ix[key][meas]
except:
    hrv.ix[key]['ApEn'] = NaN
#Sample Ent
try:
    Results = samp_entropy_wrapper(event_type, meas, Data, Settings, Results)
    hrv.ix[key]['Sample Ent'] = Results['Sample Entropy'].ix[key][meas]
except:
    hrv.ix[key]['Sample Ent'] = NaN

    
hrv.to_csv(r"%s/%s_HRV.csv"%(Settings['Output Folder'],Settings['Label']))
end_time = t.clock()

print 'Heart Rate Varibility Analysis Complete: %s sec' %np.round((end_time- start_time), 4)
hrv

Made plots folder
Data Loaded
Rounded Sampling Rate (s/frame): 0.00025
2014-06-10rat70-ECG.txt is 7278.94975 seconds long.
Rounded Sampling Rate (s/frame): 0.00025
Transformation completed
Baseline set completed
Peak Detection completed
Burst Detection completed
Analysis Complete:  458.7132  Seconds

--------------------------------------------
Data Column Names/Keys
-----
2014-06-10rat70-ECG.txt

--------------------------------------------
Available Measurements from Peaks for further analysis:
-----
Peaks Amplitude
Intervals

--------------------------------------------
Available Measurements from Bursts for further analysis:
-----
Burst Start
Burst End
Burst Duration
Edge Event
Interburst Interval
Total Cycle Time
Peaks per Burst
Peak Amp
Peak Time
Attack
Decay
Intraburst Frequency

---------------------------
|Event Detection Complete!|
---------------------------

Heart Rate Varibility Analysis Complete: 676.847 sec


,Beats,Recording Length (s),Mean HR,Mean RR,STD RR,pNN50,rMSSD,IRRR,MADRR,TINN,HRV index,ApEn,Sample Ent,HistEnt,SD1,SD2,ULF,VLF,LF,HF,Total Power,LF/HF Ratio
2014-06-10rat70-ECG.txt,50028,7278.95,412.3782,0.1454966,0.05323405,44,0.1549293,0.06275,0.16475,0.0326547,130,NaN,NaN,0.6596976,NaN,NaN,9.612133e-06,1.827792e-05,8.877986e-05,0.0003129404,0.0004829241,0.2836957


#OPTIONAL GRAPHS AND ANALYSIS

The following blocks are optional calls to other figures and analysis

## Display Event Detection Tables
#### Display Settings used for analysis

In [ ]:
display_settings(Settings)

#### Display Summary Results for Peaks

In [ ]:
#grouped summary for peaks
Results['Peaks-Master'].groupby(level=0).describe()

#### Display Summary Results for Bursts

In [ ]:
#grouped summary for bursts
Results['Bursts-Master'].groupby(level=0).describe()

# Interactive Graphs
## Line Graphs
####One pannel, detected events
Plot one time series by calling it's name

In [4]:
#Interactive, single time series by Key
key = Settings['Label']
graph_ts(Data, Settings, Results, key)

####Two pannel
Create line plots of the raw data as well as the data analysis. 

Plots are saved by clicking the save button in the pop-up window with your graph.

    key = 'Mean1'
    start =100 
    end= 101

[Results Line Plot](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#line-plots)

In [5]:
key = Settings['Label']
start =100 #start time in seconds
end= 101#end time in seconds
results_timeseries_plot(key, start, end, Data, Settings, Results)

## Autocorrelation

Display the Autocorrelation plot of your transformed data.

Choose the start and end time in seconds. to capture whole time series, use end = -1. May be slow

    key = 'Mean1'
    start = 0 
    end = 10
    
[Autocorrelation Plot](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#autocorrelation)

In [ ]:
#autocorrelation
key = Settings['Label']
start = 0 #seconds, where you want the slice to begin
end = 1 #seconds, where you want the slice to end.
autocorrelation_plot(Data['trans'][key][start:end])
plt.show()

## Raster Plot

Shows the temporal relationship of peaks in each column. Auto scales. Display only. Intended for more than one column of data

In [ ]:
#raster
raster(Data, Results)

## Frequency Plot
Use this block to plot changes of any measurement over time. Does not support 'all'. Example:

    event_type = 'Peaks'
    meas = 'Intervals'
    key = 'Mean1'

[Frequency Plot](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#frequency-plot)

In [5]:
event_type = 'Peaks'
meas = 'Intervals'
key = Settings['Label']
frequency_plot(event_type, meas, key, Data, Settings, Results)

## Analyze Events by Measurement
Generates a line plot with error bars for a given event measurement. X axis is the names of each time series. Display Only. Intended for more than one column of data. This is not a box and whiskers plot.

    event_type = 'peaks'
    meas = 'Peaks Amplitude'
    
[Analyze Events by Measurement](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#analyze-events-by-measurement)

In [ ]:
#Get average plots, display only
event_type = 'Peaks'
meas = 'Intervals'
average_measurement_plot(event_type, meas,Results)

## Poincare Plots

Create a Poincare Plot of your favorite varible. Choose an event type (Peaks or Bursts), measurement type. Calling meas = 'All' is supported.

Plots and tables are saved automatically

Example:

    event_type = 'Bursts'
    meas = 'Burst Duration'

[More on Poincare Plots](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#poincare-plots)

####Batch Poincare

[Batch Poincare](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#batch-poincare)

In [ ]:
#Batch
event_type = 'Peaks'
meas = 'Intervals'
Results = poincare_batch(event_type, meas, Data, Settings, Results)
pd.concat({'SD1':Results['Poincare SD1'],'SD2':Results['Poincare SD2']})

####Quick Poincare Plot

Quickly call one poincare plot for display. Plot and Table are not saved automatically. Choose an event type (Peaks or Bursts), measurement type, and key. Calling meas = 'All' is not supported.

[Quick Poincare](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#quick-poincare-plots)

In [4]:
#quick
event_type = 'Peaks'
meas = 'Intervals'
key = Settings['Label']
poincare_plot(Results[event_type][key][meas])

Intervals results:
SD1 = 0.0517 s
SD2 = 0.0548 s


## Power Spectral Density
The following blocks allows you to asses the power of event measuments in the frequency domain. While you can call this block on any event measurement, it is intended to be used on interval data (or at least data with units in seconds). Reccomended:

    event_type = 'Bursts'
    meas = 'Total Cycle Time'
    key = 'Mean1'
    scale = 'raw'
    
    event_type = 'Peaks'
    meas = 'Intervals'
    key = 'Mean1'
    scale = 'raw'

Because this data is in the frequency domain, we must interpolate it in order to perform a FFT on it. Does not support 'all'.

[Power Spectral Density: Events](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#power-spectral-density)

### Events

Use the code block below to specify your settings for event measurment PSD.

In [ ]:
Settings['PSD-Event'] = Series(index = ['Hz','ULF', 'VLF', 'LF','HF','dx'])
#Set PSD ranges for power in band

Settings['PSD-Event']['hz'] = 4.0 #freqency that the interpolation and PSD are performed with.
Settings['PSD-Event']['ULF'] = 0.03 #max of the range of the ultra low freq band. range is 0:ulf
Settings['PSD-Event']['VLF'] = 0.05 #max of the range of the very low freq band. range is ulf:vlf
Settings['PSD-Event']['LF'] = 0.15 #max of the range of the low freq band. range is vlf:lf
Settings['PSD-Event']['HF'] = 0.4 #max of the range of the high freq band. range is lf:hf. hf can be no more than (hz/2)
Settings['PSD-Event']['dx'] = 10 #segmentation for the area under the curve. 

In [ ]:
event_type = 'Peaks'
meas = 'Intervals'
key = Settings['Label']
scale = 'raw'
Results = psd_event(event_type, meas, key, scale, Data, Settings, Results)
Results['PSD-Event'][key]

### Time Series

Use the settings code block to set your frequency bands to calculate area under the curve. This block is not required. band output is always in raw power, even if the graph scale is dB/Hz.

[Power Spectral Density: Signal](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#power-spectral-density-optional)

In [ ]:
#optional
Settings['PSD-Signal'] = Series(index = ['ULF', 'VLF', 'LF','HF','dx'])

#Set PSD ranges for power in band
Settings['PSD-Signal']['ULF'] = 25 #max of the range of the ultra low freq band. range is 0:ulf
Settings['PSD-Signal']['VLF'] = 75 #max of the range of the very low freq band. range is ulf:vlf
Settings['PSD-Signal']['LF'] = 150 #max of the range of the low freq band. range is vlf:lf
Settings['PSD-Signal']['HF'] = 300 #max of the range of the high freq band. range is lf:hf. hf can be no more than (hz/2) where hz is the sampling frequency
Settings['PSD-Signal']['dx'] = 2 #segmentation for integration of the area under the curve. 

Use the block below to generate the PSD graph and power in bands results (if selected). scale toggles which units to use for the graph:
    
    raw = s^2/Hz
    db = dB/Hz = 10*log10(s^2/Hz)
    
Graph and table are automatically saved in the `PSD-Signal` subfolder.

In [ ]:
scale = 'raw' #raw or db
Results = psd_signal(version = 'original', key = 'Mean1', scale = scale, 
                     Data = Data, Settings = Settings, Results = Results)
Results['PSD-Signal']

### Spectrogram

Use the block below to get the spectrogram of the signal. The frequency (y-axis) scales automatically to only show 'active' frequencies. This can take some time to run. 

    version = 'original'
    key = 'Mean1'

After transformation is run, you can call version = 'trans'. This graph is not automatically saved.

[Spectrogram](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#spectrogram)

In [ ]:
version = 'original'
key = Settings['Label']
spectogram(version, key, Data, Settings, Results)

## Descriptive Statistics

#### Moving/Sliding Averages, Standard Deviation, and Count
Generates the moving **mean**, **standard deviation**, and **count** for a given measurement across all columns of the Data in the form of a DataFrame (displayed as a table).
Saves out the dataframes of these three results automatically with the window size in the name as a .csv.
If **meas == 'All'**, then the function will loop and produce these tables for all measurements.

    event_type = 'Peaks'
    meas = 'all'
    window = 30
    
[Moving Stats](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#analyze-events-by-measurement)

In [ ]:
#Moving Stats
event_type = 'Peaks'
meas = 'Intervals'
window = 30 #seconds
Results = moving_statistics(event_type, meas, window, Data, Settings, Results)

## Entropy

### Histogram Entropy
Calculates the histogram entropy of a measurement for each column of data. Also saves the histogram of each. If meas is set to 'all', then all available measurements from the event_type chosen will be calculated iteratevely. 

If all of the samples fall in one bin regardless of the bin size, it means we have the most predictable sitution and the entropy is 0. If we have uniformly dist function, the max entropy will be 1

    event_type = 'Bursts'
    meas = 'all'

[Histogram Entropy](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#histentropy)

In [9]:
#Histogram Entropy
event_type = 'Peaks'
meas = 'Intervals'
Results = histent_wrapper(event_type, meas, Data, Settings, Results)
Results['Histogram Entropy']

,Intervals
2014-06-10rat70-ECG.txt,0.197255


## Approximate entropy
this only runs if you have pyeeg.py in the same folder as this notebook and bass.py. **WARNING: THIS FUNCTION RUNS SLOWLY**

run the below code to get the approximate entropy of any measurement or raw signal. Returns the entropy of the entire results array (no windowing). I am using the following M and R values:

    M = 2  
    R = 0.2*std(measurement)
    
these values can be modified in the source code. alternatively, you can call ap_entropy directly. supports 'all'

**Interpretation**: A time series containing many repetitive patterns has a relatively small ApEn; a less predictable process has a higher ApEn.

[Approximate Entropy in BASS](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#approximate-entropy)

[Aproximate Entropy Source](http://pyeeg.sourceforge.net/)

###Events

In [ ]:
#Approximate Entropy
event_type = 'Peaks'
meas = 'Intervals'
Results = ap_entropy_wrapper(event_type, meas, Data, Settings, Results)
Results['Approximate Entropy']

###Time Series

In [ ]:
#Approximate Entropy on raw signal
#takes a VERY long time
from pyeeg import ap_entropy

version = 'original' #original, trans, shift, or rolling
key = Settings['Label'] #Mean1 default key for one time series
start = 0 #seconds, where you want the slice to begin
end = 1 #seconds, where you want the slice to end. The absolute end is -1

ap_entropy(Data[version][key][start:end].tolist(), 2, (0.2*np.std(Data[version][key][start:end])))

### Sample Entropy
this only runs if you have pyeeg.py in the same folder as this notebook and bass.py. **WARNING: THIS FUNCTION RUNS SLOWLY**

run the below code to get the sample entropy of any measurement. Returns the entropy of the entire results array (no windowing). I am using the following M and R values:

    M = 2  
    R = 0.2*std(measurement)
    
these values can be modified in the source code. alternatively, you can call samp_entropy directly. 
Supports 'all'

[Sample Entropy in BASS](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#sample-entropy)

[Sample Entropy Source](http://pyeeg.sourceforge.net/)

###Events

In [ ]:
#Sample Entropy
event_type = 'Peaks'
meas = 'Intervals'
Results = samp_entropy_wrapper(event_type, meas, Data, Settings, Results)
Results['Sample Entropy']

###Time Series

In [ ]:
#on raw signal
#takes a VERY long time
version = 'original' #original, trans, shift, or rolling
key = Settings['Label']
start = 0 #seconds, where you want the slice to begin
end = 1 #seconds, where you want the slice to end. The absolute end is -1

samp_entropy(Data[version][key][start:end].tolist(), 2, (0.2*np.std(Data[version][key][start:end])))

# Helpful Stuff

While not completely up to date with some of the new changes, the Wiki can be useful if you have questions about some of the settings: https://github.com/drcgw/SWAN/wiki/Tutorial

# More Help?

Stuck on a particular step or function?
Try typing the function name followed by two ??. This will pop up the docstring and source code.
You can also call help() to have the notebook print the doc string.

    Example:
    analyze??
    help(analyze)

In [ ]:
help(moving_statistics)

In [ ]:
moving_statistics??

##Blank Code Block
you're still here, reading? you must be a dedicated super user!

If that is the case, then you must know how to code in Python. Use this space to get crazy with your own advanced analysis and stuff.

[Blank Code Block](https://github.com/drcgw/bass/wiki/Single-Wave-Interactive-Protocol#blank-code-block)